In [3]:
import numpy as np
import pandas as pd

sess = pd.read_csv('cinema_sessions.csv')
tit = pd.read_csv('titanic_with_labels.csv')
sess

,check_number session_start
0,0 6729 08:42:00.000
1,1 6982 15:30:00.000
2,2 8219 14:15:00.000
3,3 6116 15:00:00.000
4,4 7707 08:42:00.000
...,...
9995,9995 6239 15:30:00.000
9996,9996 7272 07:30:00.000
9997,9997 7822 07:30:00.000
9998,9998 7397 15:34:00.000
